In [1]:
from keras.models import load_model
from keras.layers import Embedding
import pandas as pd
import numpy as np
import re
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gradio as gr

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LSTM = load_model("SavedModels/biLSTM_w2v.h5")

In [3]:
data_train = pd.read_csv("Dataset/data_train.csv", encoding="utf-8")
data_test = pd.read_csv("Dataset/data_test.csv", encoding="utf-8")
print(data_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7934 entries, 0 to 7933
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Emotion  7934 non-null   object
 1   Text     7934 non-null   object
dtypes: object(2)
memory usage: 124.1+ KB
None


In [4]:
data = pd.concat([data_test, data_train])
print(data)

      Emotion                                               Text
0     sadness  I experienced this emotion when my grandfather...
1     neutral   when I first moved in , I walked everywhere ....
2       anger  ` Oh ! " she bleated , her voice high and rath...
3        fear  However , does the right hon. Gentleman recogn...
4     sadness  My boyfriend didn't turn up after promising th...
...       ...                                                ...
7929  sadness                When I felt alone and without love.
7930    anger                   Irina hung up in exasperation . 
7931  sadness  No wonder she was now inconsolable at the pros...
7932     fear  My friend had been telling me about a certain ...
7933    anger   I've created a monster ... you're a better ba...

[11327 rows x 2 columns]


In [5]:
def clean_text(data):

    # remove hashtags and @usernames
    data = re.sub(r"(#[\d\w\.]+)", '', data)
    data = re.sub(r"(@[\d\w\.]+)", '', data)
    
    # tekenization using nltk
    data = word_tokenize(data)
    
    return data

In [6]:
texts = [' '.join(clean_text(text)) for text in data.Text]
print(texts)

['I experienced this emotion when my grandfather passed away .', 'when I first moved in , I walked everywhere . But within a week , I had my purse stolen — just a block away from the police station ! Now , I always take public transportation .', '` Oh ! `` she bleated , her voice high and rather indignant .', 'However , does the right hon . Gentleman recognise that profound disquiet has been expressed about some of the proposals in the Bill , particularly the fast-track ones ?', "My boyfriend did n't turn up after promising that he was coming .", "It 's freezing .", 'That ’ s not all ! I also had to finish writing a sales report for my boss . In the end , I finished everything . I wonder what will be waiting for me tomorrow morning .', "I do n't have a warrant .", 'I guess so .', 'I was just robbed !', 'This is it ?', 'Well , do you think I have to ?', 'They seemed anxious and hesitant about leaving , as if uncertain of which direction to take .', 'I experienced anger most recently whe

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [8]:
ClassName = ['happy', 'anxious', 'anger', 'sadness', 'neutral']

In [9]:
def Predict(text):
    text = [text]
    seq = tokenizer.texts_to_sequences(text)
    pad = pad_sequences(seq, maxlen=500)
    pred = LSTM.predict(pad)
    print(ClassName[np.argmax(pred)])
    return ClassName[np.argmax(pred)]

In [10]:
iface = gr.Interface(
    fn = Predict,
    inputs = "text",
    outputs = "text",
    live = True
)

In [11]:
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://cfbbd3971655677537.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 2s 2s/step
anger


In [46]:
iface.close()

Closing server running on port: 7863
